Comenzaremos por importar los módulos necesarios

In [ ]:
import pandas as pd #Importamos las librerías necesarias y los módulos de scikit-learn que utilizaremos
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

Además, en el proceso probé otros modelos de sklearn, aunque no quedé conforme con los resultados:
GaussianNB, BernoulliNB, CategoricalNB, SVC, LogisticRegression

Ingestamos los datasets con los que trabajaremos

In [ ]:
train = pd.read_excel('E-Commerce_train.xlsx') #Ingestamos ambos archivos
test = pd.read_excel('E-Commerce_test.xlsx')

Intentaremos conocer un poco nuestros datos, y buscaremos valores faltantes o marcadamente fuera de lo común

In [ ]:
print('Train:\n',train.isnull().sum()) #Comenzando con el set Train, contamos los valores faltantes
print(train.describe())              #Observamos las estadísticas más comunes
print(train.columns)                #Listamos las columnas para una mejor comprensión
for col in train.columns: print(np.sort(train[col].unique()))   #Observamos todos los valores para cada columna
print('Test:\n',test.isnull().sum()) #Repetimos el proceso en el set Test
print(test.describe())
print(test.columns)
for col in test.columns: print(np.sort(test[col].unique()))

Del análisis exploratorio no hemos obtenido valores faltantes o marcadamente fuera de lo común, pero habiendo percibido una importante diferencia entre los valores de 'Discount_offered' de las tablas, decidimos analizarla

In [ ]:
train.groupby('Discount_offered').describe() #Observamos en este caso la relación ente el descuento y el promedio de entregas a tiempo

Hemos descubierto que aquellas ventas con descuento mayor al 10% siempre se entregaron a tiempo, pero como el dataset de testeo no tiene descuentos mayores al 10% eliminaremos dichas entradas. Además convertiremos en número las columnas con valores categóricas, para poder utilizarlas en el modelo

In [ ]:
df = train.loc[train['Discount_offered'] <= 10] #Quitamos los descuentos mayores al 10%
importance = {'low':1,'medium':2,'high':3} #Haremos diccionarios para facilitar el replace en cada columna
gender = {'F':1,'M':2}
warehouse = {'A':1,'B':2,'C':3,'D':4,'F':5}
shipment = {'Road':1,'Flight':2,'Ship':3}
num = df.replace({'Product_importance':importance,'Gender':gender,'Warehouse_block':warehouse,'Mode_of_Shipment':shipment}) #Reemplazamos los valores categóricos por códigos numéricos, para poder utilizar los datos con el modelo más tarde
num.describe() #Observamos el dataframe ya numerizado

Hemos realizado pruebas con el dataset, quitando columnas y comparando los resultados del modelo, descubriendo así que la columna que más influenciaba el modelo era 'ID', por lo que decidimos revisarla. Dado que la tarea se vuelve compleja por la diversidad de IDs, discretizaremos la columna

In [ ]:
viz = num.copy() #Hacemos una copia del dataframe  
viz['binned']=pd.cut(viz['ID'],bins=9) #Discretizamos la columna 'ID' en 'binned'
viz.groupby('binned').describe() #Observamos el dataframe agrupando valores por la columna 'binned', en busca de valores atípicos

Hemos obtenido que las entradas con ID menor a 3003 siempre se entregaron a tiempo, pero al no encontrarse valores de ID menores a 9000 en el set de testeo se eliminarán

In [ ]:
num1 = num.loc[(num['ID'] > 3002)] #Quitamos los ID menores a 3003

Comprobaremos si hay algún patrón visible en las demás variables, aunque sólo dejaremos registrado Mode_of_Shipment a modo de ejemplo

In [ ]:
sns.catplot(x='Mode_of_Shipment',hue='Reached.on.Time_Y.N',data=num1,kind="count") #Graficamos para comprobar si a lo largo de los valores de la columna cabia la relación entre 1 y 0 en nuestro target

No habiendo encontrado nuevos patrones concluimos que la EDA está terminada satisfactoriamente, y procedemos a preparar los arrays para entrenar el modelo y hacer el split entre train y test

In [ ]:
x = num1.drop(columns='Reached.on.Time_Y.N').values #Obtenemos los arrays que necesitamos del dataset, uno con las variables y el otro con el target
y = num1['Reached.on.Time_Y.N'].values
x.reshape(-1,1) #Nos aseguramos que los arrays tengan la forma necesaria 
y.reshape(-1,1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=7) #Realizamos el split

IMPORTANTE: Al realizar el split hemos establecido un random_state para poder replicar la misma distribución. Al observar las matrices de confusión los datos del target parecen balanceados, por lo que no se realizó una estratificación. El número 7 se eligió por ser mi numero de la suerte :) No obstante, en caso de replicarse el proceso en una computadora con otro sistema operativo (que no sea Linux Mint 20.3 Una) el split podría resultar diferente, por no aplicar de la misma manera random_state entre distintos OS. Por este motivo, exportaremos luego los subsets de entrenamiento y testeo como .csv

In [ ]:
a = np.hstack((x_train,y_train.reshape(-1,1))) #Rearmamos los arrays teniendo en cuenta el split
b = np.hstack((x_test,y_test.reshape(-1,1)))
adf = pd.DataFrame(a,columns=list(train.columns)) #Convertimos los arrays en dataframes
bdf = pd.DataFrame(b,columns=list(train.columns))
adf.to_csv('train_subset.csv',index=False) #Exportamos los subsets
bdf.to_csv('test_subset.csv',index=False)

En el caso que se quisiera probar el modelo con los datasets generados por el split, se deberá correr únicamente la importación de librerías y las celdas que siguen a esta. En caso contrario, se debera omitir la póxima celda.
Por fines prácticos toda la celda estará comentada, por lo que antes de ejecutarla se deberán quitar las líneas 1 y 12

In [ ]:
'''
train_subset = pd.read_csv('train_subset.csv') #Ingestamos los datos en dataframes
test_subset = pd.read_csv('test_subset.csv')
x_train = train_subset.drop(columns='Reached.on.Time_Y.N').values  #Obtenemos los arrays de entrenamiento y testeo
y_train = train_subset['Reached.on.Time_Y.N'].values
x_test = test_subset.drop(columns='Reached.on.Time_Y.N').values
y_test = test_subset['Reached.on.Time_Y.N'].values
x_train.reshape(-1,1) #Aseguramos que la forma de los arrays sea la requerida por el modelo
y_train.reshape(-1,1)
x_test.reshape(-1,1)
y_test.reshape(-1,1)
'''

Construiremos el modelo de Árbol y lo entrenaremos. Max_depth se estableció por obtener el mejor recall luego de probar con diversos valores

In [ ]:
model = DecisionTreeClassifier(max_depth=7) #Creamos el modelo

model.fit(x_train,y_train) #Entrenamos el modelo

Realizaremos las predicciones de nuestro modelo, para comprobar su funcionamiento

In [ ]:
y_train_pred = model.predict(x_train) #Realizamos las predicciones
y_test_pred = model.predict(x_test)

matrix1 = confusion_matrix(y_train, y_train_pred) #Armamos las matrices de confusión para ambos subsets
matrix2 = confusion_matrix(y_test,y_test_pred)

print('Matrix1:\n',matrix1,'\nMatrix2:\n',matrix2) #Imprimimos las matrices

Hemos utilizado el modelo para predecir, dando como resultado un buen recall, aunque predice demasiados falsos positivos en el set de entrenamiento

Realizaremos ahora en el set de testeo la misma normalización que realizamos en el set de entrenamiento

In [ ]:
testing = test.replace({'Product_importance':importance,'Gender':gender,'Warehouse_block':warehouse,'Mode_of_Shipment':shipment})
#Hacemos la misma normalización que en el set anterior, sin necesidad de rehacer los diccionarios, y lo imprimiremos para controlar

Por último, haremos la predicción sobre el set de testeo y exportaremos los resultados en el formato solicitado

In [ ]:
resultados = model.predict(testing) #Realizamos las predicciones
res = pd.DataFrame(resultados,columns=['pred']) #Convertimos los resultados a un dataframe
res.to_csv('UMazzucco.csv',index=False) #Exportamos los resultados

Finalmente, comprobaremos que nuestro modelo no predice cantidades astronómicas de positivos

In [ ]:
print('El modelo predijo',float(res.mean()*int(len(res))),'valores positivos') #Imprimimos un mensaje con la cantidad de valores 1 en el .csv de salida